### 多目标遗传算法（Multi-Objective Genetic Algorithm, MOGA）

#### 一、定义  
多目标遗传算法（MOGA）是一类**专为求解多目标优化问题（MOP）而设计的进化算法**，其目标是同时优化两个或以上相互冲突的目标函数，并输出一组**Pareto 最优解（Pareto Optimal Set）**，构成**Pareto 前沿（Pareto Front）**。  
与单目标 GA 不同，MOGA **不追求单一“最优解”**，而是寻找**权衡解集**——其中任一解都无法在不损害至少一个其他目标的前提下改进某一目标。

> **核心挑战**：  
> - **支配关系替代适应度标量**：个体间通过 **Pareto 支配（Dominance）** 关系比较，而非单一数值；  
> - **收敛性与多样性需兼顾**：既要逼近真实 Pareto 前沿（收敛性），又要均匀覆盖其全域（分布性）；  
> - **高维退化问题**：当目标数 ≥ 4 时，传统多样性保持机制（如拥挤距离）失效。

---

#### 二、基本流程（以 NSGA-II/III 为范式）

MOGA 继承 GA 的种群迭代框架，但在 **环境选择（Environmental Selection）** 阶段引入多目标特有机制：

##### 1. **初始化（Initialization）**
- 随机生成初始种群 $ P_0 $（大小为 *N*），个体编码方式依问题而定（常为实数向量）；
- **关键要求**：初始种群需具备足够多样性，以覆盖潜在 Pareto 前沿区域。

##### 2. **评估（Evaluation）**
- 对每个个体 $ x_i $，计算其 **目标向量** $ \mathbf{F}(x_i) = \big(f_1(x_i), f_2(x_i), \dots, f_M(x_i)\big) $；
- **不计算标量适应度**——所有目标值独立保留，用于后续支配判断。

##### 3. **非支配排序（Non-dominated Sorting）**
- 将种群分层为多个**非支配前沿（Fronts）** $ F_1, F_2, \dots $：  
  - $ F_1 $：不被任何其他解支配的解（Pareto 最优近似）；  
  - $ F_2 $：在移除 $ F_1 $ 后不被支配的解；依此类推。  
- **支配定义**：解 $ x $ 支配 $ y $（记为 $ x \prec y $）当且仅当：  
  $$
  \forall m \in \{1,\dots,M\},\ f_m(x) \leq f_m(y) \quad \text{且} \quad \exists m:\ f_m(x) < f_m(y)
  $$

##### 4. **环境选择（Environmental Selection）**
- 合并父代 $ P_t $ 与子代 $ Q_t $ 得候选集 $ R_t = P_t \cup Q_t $（大小 2*N*）；  
- 按非支配序依次选取前沿 $ F_1, F_2, \dots $ 加入新种群 $ P_{t+1} $；  
- **当加入某前沿 $ F_k $ 会导致种群超员时，启动多样性保持机制**：  
  | 算法       | 多样性机制                     | 适用目标维数 |
  |------------|------------------------------|------------|
  | **NSGA-II** | **拥挤距离（Crowding Distance）**：<br>估计解在目标空间的局部密度，距离越大越稀疏 | *M* = 2~3 |
  | **NSGA-III** | **参考点关联（Reference-point Association）**：<br>① 预定义均匀参考点；<br>② 将解关联至最近参考点；<br>③ 优先保留关联解少/垂直距离小的解 | *M* ≥ 4 |

##### 5. **交叉与变异（Crossover & Mutation）**
- 与单目标 GA 相同（如 SBX + 多项式变异）；  
- **注意**：这些操作在**决策空间**进行，不影响多目标特有选择逻辑。

重复上述流程，直至终止条件，最终 $ F_1 $ 即为近似 Pareto 最优解集。
